# EEG Analysis - Interactive Exploration

**Purpose**: Run and modify the analysis interactively

**When to use this**:
- Modify analysis parameters
- Experiment with different frequency bands
- Generate custom visualizations
- Explore specific conditions in detail

**For results only**: See `../ANALYSIS_SUMMARY.md`

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import Image, display

# Import analysis modules
from analysis.eeg_analysis import EEGAnalyzer

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Run Analysis

This runs the full pipeline: preprocessing, epoching, power calculation, statistics.

In [ ]:
# Initialize analyzer
analyzer = EEGAnalyzer(
    data_dir='../data/cleaned',
    metadata_path='../data/experiment_metadata.json'
)

# Run analysis
results = analyzer.analyze_all_subjects()
print("\nAnalysis complete!")

## 2. Statistical Tests

In [ ]:
# Run statistical tests
stats_results = analyzer.statistical_analysis()

# Create summary table
categories = analyzer.aggregate_by_category()
summary_data = []
for cat_name, cat_data in categories.items():
    if len(cat_data) > 0:
        mu_powers = [np.mean(item['mu_power']) for item in cat_data]
        beta_powers = [np.mean(item['beta_power']) for item in cat_data]
        
        summary_data.append({
            'Condition': cat_name.replace('_', ' ').title(),
            'N': len(cat_data),
            'Mu Mean': f"{np.mean(mu_powers):.2e}",
            'Mu Std': f"{np.std(mu_powers):.2e}",
            'Beta Mean': f"{np.mean(beta_powers):.2e}",
            'Beta Std': f"{np.std(beta_powers):.2e}"
        })

summary_df = pd.DataFrame(summary_data)
print("\nSummary Statistics:")
summary_df

## 3. Visualizations

In [ ]:
# Generate plots
analyzer.plot_summary(output_dir='../results')

# Display results
print("Band Power Comparison:")
display(Image(filename='../results/band_power_comparison.png'))

In [ ]:
print("Time-Frequency Analysis:")
display(Image(filename='../results/time_frequency_plots.png'))

## 4. Custom Analysis (Modify as Needed)

Example: Plot specific condition and channel

In [ ]:
# Example: Extract data for voluntary right hand, channel C3
for subject_id, subject_data in results.items():
    for condition, data in subject_data.items():
        if 'Right hand voluntary' in condition:
            epochs = data['epochs']
            print(f"{subject_id} - {condition}")
            print(f"  Epochs: {len(epochs)}")
            print(f"  Duration: {epochs.times[-1]:.1f}s per epoch")
            
            # Plot one epoch
            epochs[0].plot(picks=['C3'], scalings='auto')
            break
    break

## 5. Experiment with Parameters

Try modifying frequency bands or epoch parameters:

In [ ]:
# Example: Custom frequency band analysis
from scipy import signal

# Define custom bands
alpha_band = (8, 12)
low_beta_band = (12, 20)
high_beta_band = (20, 30)

# Analyze one condition
condition_name = 'Right hand voluntary movement'
for subject_data in results.values():
    if condition_name in subject_data:
        epochs = subject_data[condition_name]['epochs']
        data = epochs.get_data()
        
        # Compute custom band power
        for band_name, (fmin, fmax) in [('Alpha', alpha_band), 
                                          ('Low Beta', low_beta_band),
                                          ('High Beta', high_beta_band)]:
            powers = []
            for epoch in data:
                for ch in epoch:
                    freqs, psd = signal.welch(ch, fs=200)
                    mask = (freqs >= fmin) & (freqs <= fmax)
                    powers.append(np.mean(psd[mask]))
            
            print(f"{band_name} ({fmin}-{fmax} Hz): {np.mean(powers):.2e} ± {np.std(powers):.2e}")
        break

## Next Steps

- **Full results & interpretation**: See `../ANALYSIS_SUMMARY.md`
- **Modify code**: Edit cells above and rerun
- **Save custom plots**: Use `plt.savefig()`
- **Export analysis**: Use `analyzer.results` dictionary